In [1]:
# # one time example
# # URL of the webpage to scrape
# url = "https://www.pro-football-reference.com/boxscores/202409050kan.htm"

# # Send a GET request to the webpage
# response = requests.get(url)

# # Parse the content of the request with BeautifulSoup
# soup = BeautifulSoup(response.content, "html.parser")


# table = soup.find('table', id="player_offense")

# add_df = pd.read_html(str(table))[0]

In [1]:
import pandas as pd
import numpy as np
import time
import nflscraPy
import requests
from bs4 import BeautifulSoup
import pandas as pd
import warnings
from datetime import datetime
import os

In [2]:
warnings.filterwarnings('ignore')

In [3]:
path = os.getcwd()

# Current Season gamelogs

In [4]:
current_date = datetime.now()
print("Current Date:", current_date)
year = current_date.year
month = current_date.month

# If current month is between January and May (inclusive), add 1 to the year
if 1 <= month <= 5:
    new_year = year + 1
else:
    new_year = year

print("Adjusted Year:", new_year)


Current Date: 2024-10-08 15:46:02.847868
Adjusted Year: 2024


In [5]:
season_gamelogs = nflscraPy._gamelogs(
    new_year
)

~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
Processing Season Gamelogs For: https://www.pro-football-reference.com/years/2024/games.htm
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~


In [6]:
completed_games = season_gamelogs[season_gamelogs['status'] == 'closed']

In [8]:
# gamelog_statistics = nflscraPy._gamelog_statistics(
#     season_gamelogs['boxscore_stats_link'][0]
# )
# gamelog_statistics

# Load in each game stats

In [7]:
# Check if data has already been loaded previously
try:
    current_stats = pd.read_csv(path + f"\data\{new_year}_stats.csv")
    current_games = list(set(current_stats['boxscore_stats_link']))
    print(f"{new_year} stats loaded in")
except:
    print(f"{new_year} stats file does not exist")
    current_stats = pd.DataFrame()
    current_games = []

2024 stats file does not exist


In [8]:
df = pd.DataFrame()
num_games = len(completed_games)
additional_games = [game for game in completed_games['boxscore_stats_link'] if game not in current_games]
game = 1
for url in additional_games:
    # Send a GET request to the webpage
    response = requests.get(url)
    # Parse the content of the request with BeautifulSoup
    soup = BeautifulSoup(response.content, "html.parser")
    table = soup.find('table', id="player_offense")
    add_df = pd.read_html(str(table))[0]
    add_df['boxscore_stats_link'] = url
    df = pd.concat([df, add_df])
    time.sleep(5)
    print(f"Additional games completed: {game} of {num_games}")
    game +=1



Additional games completed: 1 of 78
Additional games completed: 2 of 78
Additional games completed: 3 of 78
Additional games completed: 4 of 78
Additional games completed: 5 of 78
Additional games completed: 6 of 78
Additional games completed: 7 of 78
Additional games completed: 8 of 78
Additional games completed: 9 of 78
Additional games completed: 10 of 78
Additional games completed: 11 of 78
Additional games completed: 12 of 78
Additional games completed: 13 of 78
Additional games completed: 14 of 78
Additional games completed: 15 of 78
Additional games completed: 16 of 78
Additional games completed: 17 of 78
Additional games completed: 18 of 78
Additional games completed: 19 of 78
Additional games completed: 20 of 78
Additional games completed: 21 of 78
Additional games completed: 22 of 78
Additional games completed: 23 of 78
Additional games completed: 24 of 78
Additional games completed: 25 of 78
Additional games completed: 26 of 78
Additional games completed: 27 of 78
Additional

In [9]:
if len(df)>0:
    df.columns = ['_'.join(col).strip() for col in df.columns.values]
    
    df.columns = ['Player','Team'] + list(df.columns[2:])
    
    df = df.loc[~df['Player'].isin([np.nan, 'Player'])]
    
    stats = pd.merge(df, completed_games, left_on ='boxscore_stats_link_', right_on='boxscore_stats_link', how='left')
    
    final_stats = pd.concat([current_stats,stats], axis=0)

    final_stats.to_csv(path + f"\data\{new_year}_stats.csv")
else:
    print(f"All games have been updated for {new_year} stats")

# Historical load

In [14]:
def historical_data(year):
    # Load in gamelogs of season
    season_gamelogs = pd.read_csv(path+f"\historical-gamelogs\Season-{year}.csv")
    completed_games = season_gamelogs[season_gamelogs['status'] == 'closed']
    
    df = pd.DataFrame()
    num_games = len(completed_games)

    try:
        current_stats = pd.read_csv(path + f"\data\{year}_stats.csv")
        current_games = list(set(current_stats['boxscore_stats_link']))
        print(f"{year} stats loaded in")
    except:
        print(f"{year} stats file does not exist")
        current_stats = pd.DataFrame()
        current_games = []
        
    additional_games = [game for game in completed_games['boxscore_stats_link'] if game not in current_games]
    game = 1
    print(f"Season: {year}")
    for url in additional_games:
        try:
            # Send a GET request to the webpage
            response = requests.get(url)
            # Parse the content of the request with BeautifulSoup
            soup = BeautifulSoup(response.content, "html.parser")
            table = soup.find('table', id="player_offense")
            add_df = pd.read_html(str(table))[0]
            add_df['boxscore_stats_link'] = url
            df = pd.concat([df, add_df])
            time.sleep(5)
            print(f"Additional games completed: {game} of {num_games}")
            game +=1
        except:
            print(f"{url} could not be processed")
            time.sleep(5)
            pass
        
    if len(df)>0:
        df.columns = ['_'.join(col).strip() for col in df.columns.values]
        df.columns = ['Player','Team'] + list(df.columns[2:])
        df = df.loc[~df['Player'].isin([np.nan, 'Player'])]
        
        stats = pd.merge(df, completed_games, left_on ='boxscore_stats_link_', right_on='boxscore_stats_link', how='left')
        final_stats = pd.concat([current_stats,stats], axis=0)
        final_stats.to_csv(path + f"\data\{year}_stats.csv")
    else:
        print(f"All games have been updated for {year} stats")

    return final_stats

In [15]:
stats_2018 = historical_data(2018)

2018 stats file does not exist
Season: 2018
Additional games completed: 1 of 267
Additional games completed: 2 of 267
Additional games completed: 3 of 267
Additional games completed: 4 of 267
Additional games completed: 5 of 267
Additional games completed: 6 of 267
Additional games completed: 7 of 267
Additional games completed: 8 of 267
Additional games completed: 9 of 267
Additional games completed: 10 of 267
Additional games completed: 11 of 267
Additional games completed: 12 of 267
Additional games completed: 13 of 267
Additional games completed: 14 of 267
Additional games completed: 15 of 267
Additional games completed: 16 of 267
Additional games completed: 17 of 267
Additional games completed: 18 of 267
Additional games completed: 19 of 267
Additional games completed: 20 of 267
Additional games completed: 21 of 267
Additional games completed: 22 of 267
Additional games completed: 23 of 267
Additional games completed: 24 of 267
Additional games completed: 25 of 267
Additional game

In [16]:
stats_2019 = historical_data(2019)
stats_2020 = historical_data(2020)
stats_2021 = historical_data(2021)
stats_2022 = historical_data(2022)
stats_2023 = historical_data(2023)

2019 stats file does not exist
Season: 2019
Additional games completed: 1 of 267
Additional games completed: 2 of 267
Additional games completed: 3 of 267
Additional games completed: 4 of 267
Additional games completed: 5 of 267
Additional games completed: 6 of 267
Additional games completed: 7 of 267
Additional games completed: 8 of 267
Additional games completed: 9 of 267
Additional games completed: 10 of 267
Additional games completed: 11 of 267
Additional games completed: 12 of 267
Additional games completed: 13 of 267
Additional games completed: 14 of 267
Additional games completed: 15 of 267
Additional games completed: 16 of 267
Additional games completed: 17 of 267
Additional games completed: 18 of 267
Additional games completed: 19 of 267
Additional games completed: 20 of 267
Additional games completed: 21 of 267
Additional games completed: 22 of 267
Additional games completed: 23 of 267
Additional games completed: 24 of 267
Additional games completed: 25 of 267
Additional game